<a href="https://colab.research.google.com/github/s-mostafa-a/COVID-19-CT-Seg-Benchmark/blob/master/benchmark_simple_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://zenodo.org/record/3870441/files/01_Task1_Prediction.zip?download=1

--2020-06-10 18:34:38--  https://zenodo.org/record/3870441/files/01_Task1_Prediction.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36704507 (35M) [application/octet-stream]
Saving to: ‘01_Task1_Prediction.zip?download=1’

01_Task1_Prediction 100%[===================>]  35.00M  7.06MB/s    in 5.0s    

2020-06-10 18:34:45 (7.06 MB/s) - ‘01_Task1_Prediction.zip?download=1’ saved [36704507/36704507]



In [0]:
!mv ./01_Task1_Prediction.zip\?download\=1 ./results.zip

In [0]:
!mkdir results
!unzip results.zip -d ./results

Archive:  results.zip
   creating: ./results/Task100_COVIDLung/
   creating: ./results/Task100_COVIDLung/fold0/
  inflating: ./results/Task100_COVIDLung/fold0/coronacases_001.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/coronacases_002.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/coronacases_003.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/coronacases_004.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/coronacases_005.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/coronacases_006.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/coronacases_009.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/coronacases_010.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/radiopaedia_10_85902_1.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/radiopaedia_10_85902_3.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/radiopaedia_14_85914_0.nii.gz  
  inflating: ./results/Task100_COVIDLung/fold0/radiopaedia_27_86410_0.nii.g

In [0]:
!wget https://zenodo.org/record/3757476/files/COVID-19-CT-Seg_20cases.zip?download=1


--2020-06-10 19:10:53--  https://zenodo.org/record/3757476/files/COVID-19-CT-Seg_20cases.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1105395143 (1.0G) [application/octet-stream]
Saving to: ‘COVID-19-CT-Seg_20cases.zip?download=1’

COVID-19-CT-Seg_20c 100%[===================>]   1.03G  7.11MB/s    in 2m 36s  

2020-06-10 19:13:30 (6.77 MB/s) - ‘COVID-19-CT-Seg_20cases.zip?download=1’ saved [1105395143/1105395143]



In [0]:
!mv ./COVID-19-CT-Seg_20cases.zip\?download\=1 dataset.zip
!mkdir ground_truth
!unzip dataset.zip -d ground_truth

Archive:  dataset.zip
  inflating: ground_truth/coronacases_001.nii.gz  
  inflating: ground_truth/coronacases_002.nii.gz  
  inflating: ground_truth/coronacases_003.nii.gz  
  inflating: ground_truth/coronacases_004.nii.gz  
  inflating: ground_truth/coronacases_005.nii.gz  
  inflating: ground_truth/coronacases_006.nii.gz  
  inflating: ground_truth/coronacases_007.nii.gz  
  inflating: ground_truth/coronacases_008.nii.gz  
  inflating: ground_truth/coronacases_009.nii.gz  
  inflating: ground_truth/coronacases_010.nii.gz  
  inflating: ground_truth/radiopaedia_10_85902_1.nii.gz  
  inflating: ground_truth/radiopaedia_10_85902_3.nii.gz  
  inflating: ground_truth/radiopaedia_14_85914_0.nii.gz  
  inflating: ground_truth/radiopaedia_27_86410_0.nii.gz  
  inflating: ground_truth/radiopaedia_29_86490_1.nii.gz  
  inflating: ground_truth/radiopaedia_29_86491_1.nii.gz  
  inflating: ground_truth/radiopaedia_36_86526_0.nii.gz  
  inflating: ground_truth/radiopaedia_40_86625_0.nii.gz  
  in

In [0]:
!git clone https://github.com/s-mostafa-a/COVID-19-CT-Seg-Benchmark
!mv ./COVID-19-CT-Seg-Benchmark ./master

Cloning into 'COVID-19-CT-Seg-Benchmark'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 409 (delta 52), reused 16 (delta 6), pack-reused 315
Receiving objects: 100% (409/409), 1.50 MiB | 1.63 MiB/s, done.
Resolving deltas: 100% (238/238), done.


In [0]:

import numpy as np
import nibabel as nb
import os
from collections import OrderedDict
import pandas as pd
import sys
sys.path.append('..')

from content.master.utils.SurfaceDice import compute_surface_distances, compute_surface_dice_at_tolerance, \
    compute_dice_coefficient


join = os.path.join

seg_path = './results/Task100_COVIDLung/fold0'
gt_path = './ground_truth'
save_path = './'
save_name = 'Task_num_SegMetric.csv'

filenames = os.listdir(seg_path)
filenames.sort()
num_labels = np.max(nb.load(join(gt_path, filenames[0])).get_fdata())

seg_metrics = OrderedDict()
seg_metrics['Name'] = list()

if num_labels == 1:
    seg_metrics['LesionDSC'] = list()
    seg_metrics['LesionNSD-3mm'] = list()

elif num_labels == 2:
    seg_metrics['L-lungDSC'] = list()
    seg_metrics['L-lung-1mm'] = list()

    seg_metrics['R-lungDSC'] = list()
    seg_metrics['R-lung-1mm'] = list()
else:
    seg_metrics['L-lungDSC'] = list()
    seg_metrics['L-lung-1mm'] = list()

    seg_metrics['R-lungDSC'] = list()
    seg_metrics['R-lung-1mm'] = list()

    seg_metrics['LesionDSC'] = list()
    seg_metrics['LesionNSD-3mm'] = list()

for name in filenames:
    seg_metrics['Name'].append(name)
    # load grond truth and segmentation
    gt_nii = nb.load(join(gt_path, name))
    case_spacing = gt_nii.header.get_zooms()
    gt_data = np.uint8(gt_nii.get_fdata())
    seg_data = nb.load(join(seg_path, name)).get_fdata()

    labels = np.unique(gt_data)[1:]
    labels = labels.tolist()

    if num_labels == 1:  # Lesion
        surface_distances = compute_surface_distances(gt_data == 1, seg_data == 1, case_spacing)
        seg_metrics['LesionDSC'].append(compute_dice_coefficient(gt_data == 1, seg_data == 1))
        seg_metrics['LesionNSD-3mm'].append(
            compute_surface_dice_at_tolerance(surface_distances, 3))

    elif num_labels == 2:  # left and right lung
        surface_distances = compute_surface_distances(gt_data == 1, seg_data == 1, case_spacing)
        seg_metrics['L-lungDSC'].append(compute_dice_coefficient(gt_data == 1, seg_data == 1))
        seg_metrics['L-lung-1mm'].append(compute_surface_dice_at_tolerance(surface_distances, 1))

        surface_distances = compute_surface_distances(gt_data == 2, seg_data == 2, case_spacing)
        seg_metrics['R-lungDSC'].append(compute_dice_coefficient(gt_data == 2, seg_data == 2))
        seg_metrics['R-lung-1mm'].append(compute_surface_dice_at_tolerance(surface_distances, 1))

    else:  # left lung, right lung and infections
        surface_distances = compute_surface_distances(gt_data == 1, seg_data == 1, case_spacing)
        seg_metrics['L-lungDSC'].append(compute_dice_coefficient(gt_data == 1, seg_data == 1))
        seg_metrics['L-lung-1mm'].append(compute_surface_dice_at_tolerance(surface_distances, 1))

        surface_distances = compute_surface_distances(gt_data == 2, seg_data == 2, case_spacing)
        seg_metrics['R-lungDSC'].append(compute_dice_coefficient(gt_data == 2, seg_data == 2))
        seg_metrics['R-lung-1mm'].append(compute_surface_dice_at_tolerance(surface_distances, 1))

        surface_distances = compute_surface_distances(gt_data == 3, seg_data == 3, case_spacing)
        seg_metrics['LesionDSC'].append(compute_dice_coefficient(gt_data == 3, seg_data == 3))
        seg_metrics['LesionNSD-3mm'].append(
            compute_surface_dice_at_tolerance(surface_distances, 3))

dataframe = pd.DataFrame(seg_metrics)
dataframe.to_csv(join(save_path, save_name), index=False)
